In [36]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [37]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [38]:
from pprint import pprint

import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "ensemble"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

pprint(CFG)

ensemble_001


set seed: 46


{'cnn': {'execution': {'best_exp_id': 'exp_022',
                       'exp_id': 'exp_022',
                       'seed': 46},
         'optimizer': {'lr': 0.01},
         'trainer': {'device': 'cuda',
                     'direction': 'minimize',
                     'es_patience': 20,
                     'loss_window': 100000,
                     'max_epoch': 15,
                     'monitor': 'valid_MeanAbsoluteError'}},
 'dataset': {'competition_dir': '/kaggle/input/atmaCup#18_dataset',
             'train_fold_path': '/kaggle/input/cv-split/train_folds.csv'},
 'deberta': {'execution': {'best_exp_id': 'exp_018',
                           'exp_id': 'exp_018',
                           'seed': 46}},
 'ensemble': {'execution': {'best_exp_id': 'ensemble_001',
                            'exp_id': 'ensemble_001',
                            'seed': 46},
              'weights': {'cnn': 0.5, 'xgboost': 0.5}},
 'lightgbm': {'execution': {'best_exp_id': 'exp_017',
                  

In [39]:
import warnings

import numpy as np
import polars as pl

from const import TARGET_COLS

warnings.filterwarnings("ignore")

In [40]:
train = pl.read_csv(os.path.join(CFG["dataset"]["competition_dir"], "train_features.csv")).sort("ID")

In [41]:
weights = [0.3, 0.4, 0.3]

In [42]:
oof_xgb = pl.read_csv(os.path.join("/kaggle/output", CFG["lightgbm"]["execution"]["best_exp_id"], "oof.csv")).sort(
    "ID"
)
oof_cnn = pl.read_csv("/kaggle/output/exp_022/oof.csv").sort("ID")
oof_cnn2 = pl.read_csv("/kaggle/output/exp_014/oof.csv").sort("ID")

oof = oof_xgb.clone()
oof = oof.with_columns(
    [oof_xgb[c] * weights[0] + oof_cnn[c] * weights[1] + oof_cnn2[c] * weights[2] for c in TARGET_COLS]
)
oof.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))

In [43]:
sub_xgb = pl.read_csv(
    os.path.join("/kaggle/output", CFG["lightgbm"]["execution"]["best_exp_id"], "submission.csv")
).sort("ID")
sub_cnn = pl.read_csv("/kaggle/output/exp_022/submission.csv").sort("ID")
sub_cnn2 = pl.read_csv("/kaggle/output/exp_014/submission.csv").sort("ID")

sub = sub_xgb.clone()
sub = sub.with_columns(
    [sub_xgb[c] * weights[0] + sub_cnn[c] * weights[1] + sub_cnn2[c] * weights[2] for c in TARGET_COLS]
)
sub.to_pandas()[TARGET_COLS].to_csv(os.path.join(CFG["output_dir"], "submission.csv"), index=False)

In [44]:
mae = np.mean(np.abs(train[TARGET_COLS].to_numpy() - oof[[c for c in TARGET_COLS]].to_numpy()))
mae

0.19059015157662482